<h2>Opting-in to mask propogation on compatible layers</h2>
<p>Most layers don't modify the time dimension, so don't need to modify the current mask. But, they still want to be able to propagete the current mask, unchecked to the next layer. This is called an <b>opt-in behaviour</b>. Defaultly, custom layer will destroy the current mask, since the framework has no way to tell whether the mask propagation is safe.<br> To make the layer which propagate the current input mask and does not modify the time dimension, we should det <b>self.supports_masking = True</b> in the layer constructor. In this case, the default behaviour of compute_mask() method is to pass the current mask through. Here is an example, the layer is whitelisted for mask propagation.</p>

In [2]:
#Importing required libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Creating a class that extends Layer of keras.layers
class MyActivation(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(MyActivation, self).__init__(**kwargs)
        
        # Signal that the layer is safe for mask propagation
        self.supports_masking = True

    def call(self, inputs):
        return tf.nn.relu(inputs)

<p> We can use the above custom layer between mask generating layer (example embedding) and a mask consuming layer (example lstm) and it will pass the mask along sothat it reaches the mask consuming layer.</p>

In [3]:
# Creating a input layer and embedding layer with mask_zero = True
inputs = keras.Input(shape=(None,), dtype="int32")
x = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)(inputs)

x = MyActivation()(x)  # Will pass the mask along
print("Mask found:", x._keras_mask)

outputs = layers.LSTM(32)(x)  # Will receive the mask

#Creating the model using inputs and outputs
model = keras.Model(inputs, outputs)

Mask found: KerasTensor(type_spec=TensorSpec(shape=(None, None), dtype=tf.bool, name=None), name='Placeholder_1:0')


<h2>Writing layers that need mask information</h2>
<p>The layers which consumes mask, will accept a mask argument in call and use it to decide whether to skip certain steps. For writing these layers, we can simply add <b>mask = None</b> argument in the call signature. The mask associated with the inputs well be passed to our layers whenever it is available. <br>
Below is the example, where a layer computes a softmax over the time dimension (axis 1) of an input sequence, discarding masked timesteps.</p>

In [5]:
# Creating a class TemporalSoftmax extending Layer of keras.layers
class TemporalSoftmax(keras.layers.Layer):
    def call(self, inputs, mask=None):
        broadcast_float_mask = tf.expand_dims(tf.cast(mask, "float32"), -1)
        inputs_exp = tf.exp(inputs) * broadcast_float_mask
        inputs_sum = tf.reduce_sum(inputs * broadcast_float_mask, axis=1, keepdims=True)
        return inputs_exp / inputs_sum

#Creating input, embedding, Dense and output layers
inputs = keras.Input(shape=(None,), dtype="int32")
x = layers.Embedding(input_dim=10, output_dim=32, mask_zero=True)(inputs)
x = layers.Dense(1)(x)
outputs = TemporalSoftmax()(x) #Object of custom layer

#Creating the model using the above layers
model = keras.Model(inputs, outputs)

#Ininitialising the above created model using np.random
y = model(np.random.randint(0, 10, size=(32, 100)), np.random.random((32, 100, 1)))